<a href="https://colab.research.google.com/github/lehi10/TopicosEnGrafica/blob/master/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2


In [0]:
!nvcc --version


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


In [0]:
%load_ext nvcc_plugin
  

created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <iostream>

using namespace std;

#define N 9
#define xF 3
#define yF 3

__global__ void MatAdd(int A[][N],int B[][xF],int C[][N])
{
	int i = threadIdx.x;
	int j = threadIdx.y;

	int it_F_x=0;
	int it_F_y=0;

	float  sum=0.0;

	int hPadd=(xF-1)/2;
	int vPadd=(yF-1)/2;

	if((i-hPadd+1>0 && j-vPadd+1>0 ) && (i+hPadd<N && j+vPadd<N))
	{
		for(int it_x=i-hPadd; it_x<i-hPadd+xF;it_x++)
		{	
			for(int it_y=i-hPadd; it_y<i-vPadd+yF;it_y++)
			{
				sum+=(1.0*B[it_F_x][it_F_y]*A[it_x][it_y]);
				it_F_x++;
			}
			it_F_x=0;
			it_F_y++;
		}	
		C[i][j]=sum;
	}
}


int main()
{
	int A[N][N]={	{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9},
					{1,2,3,4,5,6,7,8,9}};

	int B[xF][yF]={{0,0,0},{0,1,0},{0,0,0}};

	int C[N][N];
	for(int i=0;i<N;i++)
	{
		for(int j=0;j<N;j++)
		{
			C[i][j]=0;
		}
		
	}

	int (*pA)[N], (*pB)[xF], (*pC)[N];

	

	cudaMalloc((void**)&pA, (N*N)*sizeof(int));
	cudaMalloc((void**)&pB, (xF*yF)*sizeof(int));
	cudaMalloc((void**)&pC, (N*N)*sizeof(int));



	cudaMemcpy(pA,A,(N*N)*sizeof(int),	cudaMemcpyHostToDevice);
	cudaMemcpy(pB,B,(xF*yF)*sizeof(int),cudaMemcpyHostToDevice);
	cudaMemcpy(pC,C,(N*N)*sizeof(int),	cudaMemcpyHostToDevice);

	int numBlocks=1;
	dim3 threadsPerBlock(N,N);

	
	MatAdd<<<numBlocks,threadsPerBlock>>>(pA,pB,pC);


	cudaMemcpy(C,pC,(N*N)*sizeof(int),cudaMemcpyDeviceToHost);


	int i,j;
	printf("C = \n");
	for(i=0;i<N;i++)
	{
		for(j=0;j<N;j++)
		{
			printf("%d " ,C[i][j]);
		}
		printf("\n");
	}

	cudaFree(pA);
	cudaFree(pB);
	cudaFree(pC);

	printf("\n");

	return 0;

}




'C = \n0 0 0 0 0 0 0 0 0 \n0 2 2 2 2 2 2 2 0 \n0 3 3 3 3 3 3 3 0 \n0 4 4 4 4 4 4 4 0 \n0 5 5 5 5 5 5 5 0 \n0 6 6 6 6 6 6 6 0 \n0 7 7 7 7 7 7 7 0 \n0 8 8 8 8 8 8 8 0 \n0 0 0 0 0 0 0 0 0 \n\n'

In [8]:
#%pip install pycuda

# -*- coding: utf-8 -*-
import numpy as np
from scipy import misc
import time

import cuconv as cu
import cpuconv as cp
from scipy import signal as sg


# Get numpy array from image
def from_img(fname):
    return np.asarray(misc.imread(fname, flatten=True), dtype=np.float32)


# Write numpy array to image
def to_img(m):
    return np.clip(np.absolute(m), 0, 255)


# Create a box blue kernel of radius r
def k_boxblur(r):
    return np.ones([r+(0 if r % 2 is 1 else 1), r+(0 if r % 2 is 1 else 1)])


# Normalize kernel matrix
def nrm(m):
    m = np.array(m)
    return m/np.sum(np.abs(m))


# CREATE KERNELS
k_sv = [[-1., 0., 1.], [-2., 0., 2.], [-1., 0., 1.]]
k_sh = [[-1., -2., -1.], [0., 0., 0.], [1., 2., 1.]]
k_b5 = k_boxblur(5)

# LOAD IMAGE
a = from_img('/content/1VgAqbP3_400x400.jpg').astype(np.float32)


# GPU
start = time.time()
c = cu.convolve(a, k_sv)
misc.imsave('/content/results/g_result_sv.png', to_img(c))
c = cu.convolve(a, k_sh)
misc.imsave('/content/results/g_result_sh.png', to_img(c))
c = cu.convolve(a, k_b5)
misc.imsave('/content/results/g_result_b5.png', to_img(c))
end = time.time()
print("GPU time: %.5f s" % (end-start))

# SCIPY
start = time.time()
c = sg.convolve(a, nrm(k_sv))
misc.imsave('/content/results/s_result_sv.png', to_img(c))
c = sg.convolve(a, nrm(k_sh))
misc.imsave('/content/results/s_result_sh.png', to_img(c))
c = sg.convolve(a, nrm(k_b5))
misc.imsave('/content/results/s_result_b5.png', to_img(c))
end = time.time()
print("Scipy time: %.5f s" % (end-start))

# CPU
start = time.time()
c = cp.convolve(a, k_sv)
misc.imsave('/content/results/c_result_sv.png', to_img(c))
c = cp.convolve(a, k_sh)
misc.imsave('/content/results/c_result_sh.png', to_img(c))
c = cp.convolve(a, k_b5)
misc.imsave('/content/results/c_result_b5.png', to_img(c))
end = time.time()
print("CPU time: %.5f s" % (end-start))



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated i

GPU time: 0.05666 s
Scipy time: 0.10008 s


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


CPU time: 12.05512 s


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
